In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install Sastrawi

     |████████████████████████████████| 215kB 7.0MB/s 


In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/review_tokopedia_01.csv')
df.head()

,Komentar,Skor
0,Heran deh toko pedia ini habis di transfer ber...,0
1,Semenjak update terbaru total cashback nya jad...,0
2,"Proses klaim muter2,cancel sepihak,proses lamb...",0
3,"Aplikasi belanja online paling enak..""sebelum ...",0
4,Karya Dari tangan manusia Tidak ada yang sempu...,0


In [ ]:
#import stopword
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
def pre_process(text):
    # Case Folding: Lowercase
    # Merubah format teks menjadi format huruf kecil semua (lowercase).
    text = text.lower()

    # Case Folding: Removing Number
    # Menghapus karakter angka.
    text = re.sub(r"\d+", "", text)

    # Case Folding: Removing Punctuation
    # Menghapus karakter tanda baca.
    text = text.translate(str.maketrans("","",string.punctuation))

    #Case Folding: Removing Whitespace
    #Menghapus karakter kosong.
    text = text.strip()

    
    #Separating Sentences with Split () Method
    #Fungsi split() memisahkan string ke dalam list dengan spasi sebagai pemisah jika tidak ditentukan pemisahnya.
    pisah = text.split()

    #Filtering using sastrawi
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    text =  stopword.remove(text)

    return text

df['Komentar'] = df['Komentar'].apply(lambda x:pre_process(x))
df.head()

SyntaxError: ignored

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (790, 2)
Test data size :  (198, 2)


In [ ]:
top_words = 10000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['Komentar'])
list_tokenized_train = tokenizer.texts_to_sequences(train_df['Komentar'])

max_review_length = 200
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['Skor']

In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           320032    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 373,333
Trainable params: 373,333
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
10/10 [==============================] - 5s 323ms/step - loss: 0.6914 - accuracy: 0.5017 - val_loss: 0.6687 - val_accuracy: 0.6013
Epoch 2/20
10/10 [==============================] - 2s 234ms/step - loss: 0.6745 - accuracy: 0.6332 - val_loss: 0.6594 - val_accuracy: 0.7215
Epoch 3/20
10/10 [==============================] - 2s 237ms/step - loss: 0.6568 - accuracy: 0.6979 - val_loss: 0.6420 - val_accuracy: 0.7342
Epoch 4/20
10/10 [==============================] - 2s 239ms/step - loss: 0.6253 - accuracy: 0.7379 - val_loss: 0.5723 - val_accuracy: 0.7595
Epoch 5/20
10/10 [==============================] - 2s 235ms/step - loss: 0.5179 - accuracy: 0.7709 - val_loss: 0.4768 - val_accuracy: 0.7658
Epoch 6/20
10/10 [==============================] - 2s 234ms/step - loss: 0.4010 - accuracy: 0.8027 - val_loss: 0.4343 - val_accuracy: 0.7911
Epoch 7/20
10/10 [==============================] - 2s 237ms/step - loss: 0.2879 - accuracy: 0.8733 - val_loss: 0.3749 - val_accuracy: 0.8734
Epoch 

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(test_df['Komentar'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['Skor']
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.7272727272727273
F1-score:  0.7244897959183674
Confusion matrix:


array([[73, 25],
       [29, 71]])